In [1]:
!git clone https://github.com/mindspore-lab/mindnlp.git

正克隆到 'mindnlp'...
remote: Enumerating objects: 28173, done.
remote: Counting objects: 100% (12921/12921), done.
remote: Compressing objects: 100% (2807/2807), done.
remote: Total 28173 (delta 10781), reused 10128 (delta 10113), pack-reused 15252 (from 2)
接收对象中: 100% (28173/28173), 54.91 MiB | 20.18 MiB/s, 完成.
处理 delta 中: 100% (20617/20617), 完成.


In [ ]:
%cd mindnlp
!git checkout ef64a3b83097c9578bb0d5326f905beeb5b50e1d
!bash scripts/build_and_reinstall.sh

/mnt/workspace/MindTinyRAG/mindnlp


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


注意：正在切换到 'ef64a3b83097c9578bb0d5326f905beeb5b50e1d'。

您正处于分离头指针状态。您可以查看、做试验性的修改及提交，并且您可以在切换
回一个分支时，丢弃在此状态下所做的提交而不对分支造成影响。

如果您想要通过创建分支来保留在此状态下所做的提交，您可以通过在 switch 命令
中添加参数 -c 来实现（现在或稍后）。例如：

  git switch -c <新分支名>

或者撤销此操作：

  git switch -

通过将配置变量 advice.detachedHead 设置为 false 来关闭此建议

HEAD 目前位于 ef64a3b8 fix optimizer args as same dtype on GPU (#1842)
running bdist_wheel
running build
running build_py
creating build
creating build/lib
creating build/lib/mindnlp
copying mindnlp/patch.py -> build/lib/mindnlp
copying mindnlp/configs.py -> build/lib/mindnlp
copying mindnlp/evaluate.py -> build/lib/mindnlp
copying mindnlp/amp.py -> build/lib/mindnlp
copying mindnlp/__init__.py -> build/lib/mindnlp
creating build/lib/tests
copying tests/common.py -> build/lib/tests
copying tests/__init__.py -> build/lib/tests
creating build/lib/mindnlp/data
copying mindnlp/data/__init__.py -> build/lib/mindnlp/data
creating build/lib/mindnlp/peft
copying mindnlp/peft/peft_model.py -> build/lib/mindnlp/peft
co

In [3]:
%cd ../
!pip install -r requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xtuner 0.1.23 requires lagent>=0.1.2, which is not installed.
xtuner 0.1.23 requires mmengine>=0.10.3, which is not installed.
xtuner 0.1.23 requires scikit-image, which is not installed.
lmdeploy 0.6.4 requires outlines<0.1.0, but you have outlines 0.1.11 which is incompatible.
lmdeploy 0.6.4 requires peft<=0.11.1, but you have peft 0.14.0 which is incompatible.
lmdeploy 0.6.4 requires torch<=2.4.0,>=2.0.0, but you have torch 2.5.1 which is incompatible.
lmdeploy 0.6.4 requires torchvision<=0.19.0,>=0.15.0, but you have torchvision 0.20.1 which is incompatible.
lmdeploy 0.6.4 requires triton<=3.0.0,>=2.2.0; sys_platform == "linux", but you have triton 3.1.0 which is incompatible.
mistral-common 1.5.1 requires tiktoken<0.8.0,>=0.7.0, but you have tiktoken 0.8.0 which is incompatible.

[notice] A new release of pip

# 导入相关包

In [4]:
import os
import PyPDF2
import markdown
import json
import tiktoken
from bs4 import BeautifulSoup
import re
import os
from copy import copy
from typing import Dict, List, Optional, Tuple, Union
import numpy as np
from tqdm import tqdm
from mindspore import Tensor

enc = tiktoken.get_encoding("cl100k_base")

# 2. 读取文件
1. 读取文件：读取对应文件夹下所有文件。

2. 提取内容：判断文件类型，设计提取内容方式，实现多种格式统一化处理。

3. 分块：采用基于最大 token 长度和覆盖内容的逻辑分割长文本，确保段落间的语义连续性。

In [5]:
class ReadFiles:
    """
    class to read files
    """

    def __init__(self, path: str) -> None:
        self._path = path
        self.file_list = self.get_files()
    
    @classmethod
    def read_pdf(cls, file_path: str):
        # 读取PDF文件
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
            return text

    @classmethod
    def read_markdown(cls, file_path: str):
        # 读取Markdown文件
        with open(file_path, 'r', encoding='utf-8') as file:
            md_text = file.read()
            html_text = markdown.markdown(md_text)
            # 使用BeautifulSoup从HTML中提取纯文本
            soup = BeautifulSoup(html_text, 'html.parser')
            plain_text = soup.get_text()
            # 使用正则表达式移除网址链接
            text = re.sub(r'http\S+', '', plain_text) 
            return text

    @classmethod
    def read_text(cls, file_path: str):
        # 读取文本文件
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    
    def get_files(self):
        # args：dir_path，目标文件夹路径
        file_list = []
        for filepath, dirnames, filenames in os.walk(self._path):
            # os.walk 函数将递归遍历指定文件夹
            for filename in filenames:
                # 通过后缀名判断文件类型是否满足要求
                if filename.endswith(".md"):
                    # 如果满足要求，将其绝对路径加入到结果列表
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".txt"):
                    file_list.append(os.path.join(filepath, filename))
                elif filename.endswith(".pdf"):
                    file_list.append(os.path.join(filepath, filename))
        return file_list

    def get_content(self, max_token_len: int = 600, cover_content: int = 150):
        docs = []
        # 读取文件内容
        for file in self.file_list:
            content = self.read_file_content(file)
            chunk_content = self.get_chunk(
                content, max_token_len=max_token_len, cover_content=cover_content)
            docs.extend(chunk_content)
        return docs

    @classmethod
    def get_chunk(cls, text: str, max_token_len: int = 600, cover_content: int = 150):
        chunk_text = []

        curr_len = 0
        curr_chunk = ''

        token_len = max_token_len - cover_content
        lines = text.splitlines()  # 假设以换行符分割文本为行

        for line in lines:
            line = line.replace(' ', '')
            line_len = len(enc.encode(line))
            if line_len > max_token_len:
                # 如果单行长度就超过限制，则将其分割成多个块
                num_chunks = (line_len + token_len - 1) // token_len
                for i in range(num_chunks):
                    start = i * token_len
                    end = start + token_len
                    # 避免跨单词分割
                    while not line[start:end].rstrip().isspace():
                        start += 1
                        end += 1
                        if start >= line_len:
                            break
                    curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                    chunk_text.append(curr_chunk)
                # 处理最后一个块
                start = (num_chunks - 1) * token_len
                curr_chunk = curr_chunk[-cover_content:] + line[start:end]
                chunk_text.append(curr_chunk)
                
            if curr_len + line_len <= token_len:
                curr_chunk += line
                curr_chunk += '\n'
                curr_len += line_len
                curr_len += 1
            else:
                chunk_text.append(curr_chunk)
                curr_chunk = curr_chunk[-cover_content:]+line
                curr_len = line_len + cover_content

        if curr_chunk:
            chunk_text.append(curr_chunk)

        return chunk_text

    @classmethod
    def read_file_content(cls, file_path: str):
        # 根据文件扩展名选择读取方法
        if file_path.endswith('.pdf'):
            return cls.read_pdf(file_path)
        elif file_path.endswith('.md'):
            return cls.read_markdown(file_path)
        elif file_path.endswith('.txt'):
            return cls.read_text(file_path)
        else:
            raise ValueError("Unsupported file type")



In [6]:
text = ReadFiles('./data').get_content(max_token_len=600, cover_content=150)  # 获得data目录下的所有文件内容并分割
text

['GithubApi使用\n#GithubAPI简介\nGithubAPI是Github提供的一组接口，允许用户通过编程方式与Github进行交互。通过这些接口，用户可以获取Github仓库的信息、创建和修改仓库、管理issue和pullrequest等。\nGithubAPI支持多种编程语言，包括Python、Java、Ruby等。用户可以通过这些语言编写程序，调用GithubAPI接口，实现自动化操作。\nGithubAPI功能\nGithubAPI提供了丰富的功能，包括但不限于：\n\n获取仓库信息\n创建和修改仓库\n管理issue和pullrequest\n获取用户信息\n管理用户权限\n搜索仓库和用户\n\n以下是一个简单的示例代码，演示如何使用Python调用GithubAPI获取仓库信息：\n```python\nimportrequests\ndefget_repo_info(owner,repo):\nurl=f"\nresponse=requests.get(url)\nifresponse.status_code==200:\nreturnresponse.json()\nelse:\nreturnNone\n调用函数获取仓库信息\nrepo_info=get_repo_info("octocat","Hello-World")\nifrepo_info:\nprint(repo_info)\nelse:\nprint("获取仓库信息失败")\n```\n以上代码中，我们使用requests库发送HTTP请求，获取仓库信息。如果请求成功，则返回JSON格式的仓库信息；如果请求失败，则返回None。\n需要注意的是，在使用GithubAPI时，需要先进行身份验证，可以通过在请求头中添加Authorization字段来实现。\n目录2\nGithubApi使用方法\nGithubApi是一种用于\n#GithubAPI常见问题\n在使用GithubAPI时，可能会遇到一些常见问题，以下是一些可能遇到的问题及其解决方法。\n问题1：无法获取\n',
 'GithubApi使用\n目录1\nGithubApi简介\nGithubAPI是Github提供的一组接口，用于通过编程方式\n##GithubApi使用方法\nGithubApi是一种基于RE

# 3. 设计Embedding

In [7]:
class BaseEmbeddings:
    """
    Base class for embeddings
    """

    def __init__(self, path: str, is_api: bool) -> None:
        self.path = path
        self.is_api = is_api

    def get_embedding(self, text: str, model: str) -> List[float]:
        raise NotImplementedError

    @classmethod
    def cosine_similarity(cls, vector1: List[float], vector2: List[float]) -> float:
        """
        calculate cosine similarity between two vectors
        """
        dot_product = np.dot(vector1, vector2)
        magnitude = np.linalg.norm(vector1) * np.linalg.norm(vector2)
        if not magnitude:
            return 0
        return dot_product / magnitude

In [8]:
class MindNLPEmbedding(BaseEmbeddings):
    """
    class for MindNLP embeddings
    """
    def __init__(self, path: str = 'BAAI/bge-base-zh-v1.5', is_api: bool = False) -> None:
        super().__init__(path, is_api)
        self._model = self.load_model(path)

    def get_embedding(self, text: str):
        sentence_embedding = self._model.encode([text], normalize_embeddings=True)
        return sentence_embedding

    def load_model(self, path: str):
        from mindnlp.sentence import SentenceTransformer
        model = SentenceTransformer(path)
        return model

    @classmethod
    def cosine_similarity(cls, sentence_embedding_1, sentence_embedding_2):
        """
        calculate similarity between two vectors
        """
        similarity = sentence_embedding_1 @ sentence_embedding_2.T
        return similarity

In [9]:
embedding = MindNLPEmbedding("BAAI/bge-base-zh-v1.5")
embedding._model

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.543 seconds.
Prefix dict has been built successfully.
No sentence-transformers model found with name BAAI/bge-base-zh-v1.5. Creating a new one with MEAN pooling.
998B [00:00, 316kB/s]                    
100%|██████████| 390M/390M [00:22<00:00, 18.3MB/s] 


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


100%|██████████| 366/366 [00:00<00:00, 2.96MB/s]
107kB [00:00, 915kB/s] 
429kB [00:00, 2.42MB/s]
100%|██████████| 125/125 [00:00<00:00, 902kB/s]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

# 4. 知识库设计

In [10]:
class VectorStore:
    def __init__(self, document: List[str] = ['']) -> None:
        self.document = document

    def get_vector(self, EmbeddingModel: BaseEmbeddings):
        self.vectors = []
        for doc in tqdm(self.document, desc="Calculating embeddings"):
            self.vectors.append(EmbeddingModel.get_embedding(doc))
        return self.vectors

    def persist(self, path: str = 'storage'):
        if not os.path.exists(path):
            os.makedirs(path)
        with open(f"{path}/document.json", 'w', encoding='utf-8') as f:
            json.dump(self.document, f, ensure_ascii=False)
        if self.vectors:
            # 将 numpy.ndarray 转换为列表
            vectors_list = [vector.tolist() for vector in self.vectors]
            with open(f"{path}/vectors.json", 'w', encoding='utf-8') as f:
                json.dump(vectors_list, f)

    def load_vector(self, EmbeddingModel: BaseEmbeddings, path: str = 'storage'):
        with open(f"{path}/vectors.json", 'r', encoding='utf-8') as f:
            vectors_list = json.load(f)
        with open(f"{path}/document.json", 'r', encoding='utf-8') as f:
            self.document = json.load(f)

        # 查询 EmbeddingModel 的类别
        if isinstance(EmbeddingModel, MindNLPEmbedding):
            # 将列表重新变为 numpy.ndarray
            self.vectors = [np.array(vector) for vector in vectors_list]
        else:
            self.vectors = vectors_list

    def get_similarity(self, vector1, vector2, EmbeddingModel: BaseEmbeddings):
        return EmbeddingModel.cosine_similarity(vector1, vector2)

    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1):
        # 获取查询字符串的嵌入向量
        query_vector = EmbeddingModel.get_embedding(query)

        # 计算查询向量与数据库中每个向量的相似度
        similarities = [self.get_similarity(query_vector, vector, EmbeddingModel) for vector in self.vectors]

        # 将相似度、向量和文档存储在一个列表中
        results = []
        for similarity, vector, document in zip(similarities, self.vectors, self.document):
            results.append({
                'similarity': similarity,
                'vector': vector,
                'document': document
            })
        # 按相似度从高到低排序
        results.sort(key=lambda x: x['similarity'], reverse=True)
        # 获取最相似的 k 个文档
        top_k_documents = [result['document'] for result in results[:k]]

        return top_k_documents

In [11]:
vector = VectorStore(text)
vector.get_vector(EmbeddingModel=embedding)
vector.persist(path='storage')  # 将向量和文档内容保存到storage目录下，下次再用就可以直接加载本地的数据库
vector.load_vector(EmbeddingModel=embedding, path='./storage')  # 加载本地的数据库

Calculating embeddings: 100%|██████████| 30/30 [00:53<00:00,  1.77s/it]


# 5. 大语言模型

In [12]:
class BaseModel:
    def __init__(self, path: str = '') -> None:
        self.path = path

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        pass

    def load_model(self):
        pass

In [13]:
PROMPT_TEMPLATE = dict(
    RAG_PROMPT_TEMPALTE="""使用以上下文来回答用户的问题。如果你不知道答案，请输出我不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:""",
    MindNLP_PROMPT_TEMPALTE="""先对上下文进行内容总结,再使用上下文来回答用户的问题。如果你不知道答案，请输出我不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:"""
)

In [14]:
class MindNLPChat(BaseModel):
    def __init__(self, path: str = '') -> None:
        super().__init__(path)
        self.load_model()

    def chat(self, prompt: str, history: List = [], content: str = '') -> str:
        prompt = PROMPT_TEMPLATE['MindNLP_PROMPT_TEMPALTE'].format(question=prompt, context=content)
        response, history = self.model.chat(self.tokenizer, prompt, history, max_length=512)
        return response

    def load_model(self):
        import mindspore
        from mindnlp.transformers import AutoTokenizer, AutoModelForCausalLM
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, mirror="huggingface")
        self.model = AutoModelForCausalLM.from_pretrained(self.path, ms_dtype=mindspore.float16, mirror="huggingface")

In [15]:
# 没有保存数据库
docs = ReadFiles('./data').get_content(max_token_len=600, cover_content=150)  # 获得data目录下的所有文件内容并分割
vector = VectorStore(docs)
embedding = MindNLPEmbedding("BAAI/bge-base-zh-v1.5")  # 创建EmbeddingModel
vector.get_vector(EmbeddingModel=embedding)
vector.persist(path='storage')  # 将向量和文档内容保存到storage目录下，下次再用就可以直接加载本地的数据库

vector.load_vector(EmbeddingModel=embedding, path='./storage')  # 加载本地的数据库

question = 'git如何新建分支？'

content = vector.query(question, EmbeddingModel=embedding, k=1)
print(content)
chat = MindNLPChat(path='openbmb/MiniCPM-2B-dpo-bf16')
print(chat.chat(question, [], content))

No sentence-transformers model found with name BAAI/bge-base-zh-v1.5. Creating a new one with MEAN pooling.
Calculating embeddings: 100%|██████████| 30/30 [00:52<00:00,  1.76s/it]


['远程仓库，可以使用gitpush命令。通常，这个命令后面会跟远程仓库的名称和要推送的分支名称。\nbash\ngitpush<remote-name><branch-name>\n例如，将本地的master分支推送到origin远程仓库：\nbash\ngitpushoriginmaster\n从远程仓库拉取\n从远程仓库获取最新的更改并合并到本地分支，可以使用gitpull命令。这个命令会将远程仓库的指定分支的更改拉取到当前分支。bash\ngitpull<remote-name><branch-name>\n例如，从origin远程仓库的master分支拉取最新更改：\nbash\ngitpulloriginmaster\n远程分支管理\n查看远程分支，可以使用gitbranch命令加上-r选项。\nbash\ngitbranch-r\n删除远程分支，可以使用gitpush命令加上--delete选项。\nbash\ngitpush<remote-name>--delete<branch-name>\n例如，删除origin远程仓库的feature分支：\nbash\ngitpushorigin--deletefeature\n远程仓库的协作与贡献\n协作和贡献通常涉及以下步骤：\n\nFork远程仓库。\nCloneFork后的仓库到本地。\n创建新的分支进行开发。\n完成开发后，将分支推送到自己的Fork仓库。\n']


1.09kB [00:00, 5.52MB/s]                 
100%|██████████| 1.90M/1.90M [00:00<00:00, 2.29MB/s]
5.92MB [00:00, 6.82MB/s]
100%|██████████| 414/414 [00:00<00:00, 2.98MB/s]
0.99kB [00:00, 4.68MB/s]                 
 19%|█▉        | 0.98G/5.08G [01:49<07:36, 9.63MB/s] 


Failed to download: HTTPSConnectionPool(host='cdn-lfs-us-1.hf-mirror.com', port=443): Read timed out.
Retrying... (attempt 0/5)


 20%|██        | 1.03G/5.08G [10:00<13:45:58, 87.7kB/s] 


Failed to download: HTTPSConnectionPool(host='cdn-lfs-us-1.hf-mirror.com', port=443): Read timed out.
Retrying... (attempt 1/5)


100%|██████████| 5.08G/5.08G [05:24<00:00, 13.4MB/s] 
MindSpore do not support bfloat16 dtype, we will automaticlly convert to float16
MiniCPMForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
100%|██████████| 113/113 [00:00<00:00, 677kB/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated.41. Use the `cache_position` model input instead.


要使用Git新建分支，请按照以下步骤操作：

1. 首先，确保您已经安装了Git。
2. 打开终端或命令提示符，切换到您想要创建分支的仓库。
3. 使用`git checkout`命令切换到当前分支，然后使用`git branch`命令查看当前分支及其子分支。
4. 要创建新的分支，请使用`git branch <分支名>`命令。例如，要创建一个 TypeError 分支，请运行：

```
git branch TypeError
```

5. 创建分支后，您可以使用`git push`命令将分支推送到远程仓库。例如不畅：

```
git push origin TypeError
```

这样，您就成功创建了一个新的


In [16]:
# 保存数据库之后
vector = VectorStore()
vector.load_vector(EmbeddingModel=embedding, path='./storage')  # 加载本地的数据库
question = 'git如何新建分支？'
content = vector.query(question, EmbeddingModel=embedding, k=3)[0]
print(content)
chat = MindNLPChat(path='openbmb/MiniCPM-2B-dpo-bf16')
print(chat.chat(question, [], content))

远程仓库，可以使用gitpush命令。通常，这个命令后面会跟远程仓库的名称和要推送的分支名称。
bash
gitpush<remote-name><branch-name>
例如，将本地的master分支推送到origin远程仓库：
bash
gitpushoriginmaster
从远程仓库拉取
从远程仓库获取最新的更改并合并到本地分支，可以使用gitpull命令。这个命令会将远程仓库的指定分支的更改拉取到当前分支。bash
gitpull<remote-name><branch-name>
例如，从origin远程仓库的master分支拉取最新更改：
bash
gitpulloriginmaster
远程分支管理
查看远程分支，可以使用gitbranch命令加上-r选项。
bash
gitbranch-r
删除远程分支，可以使用gitpush命令加上--delete选项。
bash
gitpush<remote-name>--delete<branch-name>
例如，删除origin远程仓库的feature分支：
bash
gitpushorigin--deletefeature
远程仓库的协作与贡献
协作和贡献通常涉及以下步骤：

Fork远程仓库。
CloneFork后的仓库到本地。
创建新的分支进行开发。
完成开发后，将分支推送到自己的Fork仓库。



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


要使用Git新建分支， vitaminB 可以这样回答：

1. 首先，确保已经安装了Git。
2. 打开终端或命令提示符，然后输入以下命令以创建一个新的分支：

   ```
   git checkout -b <branch-name>
   ```

   例如，如果你想创建一个名为 "new-feature" 的分支，可以输入：

   ```
   git checkout trello:new-feature
   ```

   这里的 "trello" 是你使用的远程仓库的分支名称，你需要根据实际情况修改。

3. 创建完成后，你可以通过 `git branch` 命令查看分支列表，并确认新分支已创建。

4. 如果你需要切换到新分支，可以输入以下命令：

   ```
   git checkout <branch-


# 6. Rerank

In [17]:
class BaseReranker:
    """
    Base class for reranker
    """

    def __init__(self, path: str) -> None:
        self.path = path

    def rerank(self, text: str, content: List[str], k: int) -> List[str]:
        raise NotImplementedError

In [18]:
class MindNLPReranker(BaseReranker):
    """
    class for MindNLP reranker
    """

    def __init__(self, path: str = 'BAAI/bge-reranker-base') -> None:
        super().__init__(path)
        self._model= self.load_model(path)

    def rerank(self, text: str, content: List[str], k: int) -> List[str]:
        query_embedding = self._model.encode(text, normalize_embeddings=True)
        sentences_embedding = self._model.encode(sentences=content, normalize_embeddings=True)
        similarity = query_embedding @ sentences_embedding.T
        # 获取按相似度排序后的索引
        ranked_indices = np.argsort(similarity)[::-1]  # 按相似度降序排序
        # 选择前 k 个最相关的候选内容
        top_k_sentences = [content[i] for i in ranked_indices[:k]]
        return top_k_sentences

    def load_model(self, path: str):
        from mindnlp.sentence import SentenceTransformer
        model = SentenceTransformer(path)
        return model

In [19]:
# 创建RerankerModel
reranker = MindNLPReranker('BAAI/bge-reranker-base')

vector = VectorStore()
vector.load_vector(EmbeddingModel=embedding, path='./storage')  # 加载本地的数据库

question = 'git如何新建分支？'

# 从向量数据库中查询出最相似的3个文档
content = vector.query(question, EmbeddingModel=embedding, k=3)
print(content)

No sentence-transformers model found with name BAAI/bge-reranker-base. Creating a new one with MEAN pooling.
799B [00:00, 2.01MB/s]                   
100%|██████████| 1.04G/1.04G [02:17<00:00, 8.09MB/s]  
Some weights of XLMRobertaModel were not initialized from the model checkpoint at BAAI/bge-reranker-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 443/443 [00:00<00:00, 3.41MB/s]
100%|██████████| 4.83M/4.83M [00:01<00:00, 2.58MB/s]
16.3MB [00:01, 12.5MB/s]
100%|██████████| 279/279 [00:00<00:00, 2.08MB/s]


['远程仓库，可以使用gitpush命令。通常，这个命令后面会跟远程仓库的名称和要推送的分支名称。\nbash\ngitpush<remote-name><branch-name>\n例如，将本地的master分支推送到origin远程仓库：\nbash\ngitpushoriginmaster\n从远程仓库拉取\n从远程仓库获取最新的更改并合并到本地分支，可以使用gitpull命令。这个命令会将远程仓库的指定分支的更改拉取到当前分支。bash\ngitpull<remote-name><branch-name>\n例如，从origin远程仓库的master分支拉取最新更改：\nbash\ngitpulloriginmaster\n远程分支管理\n查看远程分支，可以使用gitbranch命令加上-r选项。\nbash\ngitbranch-r\n删除远程分支，可以使用gitpush命令加上--delete选项。\nbash\ngitpush<remote-name>--delete<branch-name>\n例如，删除origin远程仓库的feature分支：\nbash\ngitpushorigin--deletefeature\n远程仓库的协作与贡献\n协作和贡献通常涉及以下步骤：\n\nFork远程仓库。\nCloneFork后的仓库到本地。\n创建新的分支进行开发。\n完成开发后，将分支推送到自己的Fork仓库。\n', 'checkout<branch_name>\n这将使当前工作目录切换到名为<branch_name>的分支上。\n合并分支\n要将一个分支的更改合并到当前分支，可以使用以下命令：\nbash\ngitmerge<branch_name>\n这将把名为<branch_name>的分支合并到当前分支上。\n解决冲突\n在合并分支时，如果发生冲突，需要手动解决冲突。可以通过编辑文件来解决冲突，然后使用以下命令标记文件为已解决冲突的状态：bash\ngitadd<file_name>\n解决完所有冲突后，可以继续合并分支。\n以上是关于Git分支管理的基本操作。\n远程仓库\n添加远程仓库\n要将本地仓库与远程仓库关联，可以使用以下命令：\nbash\ngitremoteaddorigin远程仓库地址\n其中，origin是远程仓库的别名，可以

In [20]:
# 从一阶段查询结果中用Reranker再次筛选出最相似的2个文档
rerank_content = reranker.rerank(question, content, k=2)
print(rerank_content)

['是用来隔离开发工作的。每个分支都是一个独立的开发环境，互不影响。分支可以很方便地被创建和合并，因此许多开发者使用分支来进行特性开发、修复bug或者尝试新想法。\nGit的一个核心概念是几乎所有操作都是本地执行的，分支也不例外。这意味着你在本地创建或切换分支，不需要与远程仓库进行通信。\n创建与合并分支\n在Git中创建新分支可以使用gitbranch命令，合并分支则使用gitmerge命令。```bash\n创建新分支\ngitbranch\n切换到新分支\ngitcheckout\n创建新分支并立即切换到该分支\ngitcheckout-b\n合并指定分支到当前分支\ngitmerge\n```\n分支策略\n合理的分支策略可以帮助团队更有效地协作。一种常见的策略是GitFlow，它定义了一个围绕项目发布的分支模型，包括功能分支、发布分支、维护分支等。\n另一种策略是GitHubFlow，它更加简单灵活，适合持续交付的项目。在GitHubFlow中，master分支通常是稳定的，并且随时可以部署。所有新的开发都在基于master的特性分支上进行，一旦完成就可以合并回master。\n解决冲突\n', 'checkout<branch_name>\n这将使当前工作目录切换到名为<branch_name>的分支上。\n合并分支\n要将一个分支的更改合并到当前分支，可以使用以下命令：\nbash\ngitmerge<branch_name>\n这将把名为<branch_name>的分支合并到当前分支上。\n解决冲突\n在合并分支时，如果发生冲突，需要手动解决冲突。可以通过编辑文件来解决冲突，然后使用以下命令标记文件为已解决冲突的状态：bash\ngitadd<file_name>\n解决完所有冲突后，可以继续合并分支。\n以上是关于Git分支管理的基本操作。\n远程仓库\n添加远程仓库\n要将本地仓库与远程仓库关联，可以使用以下命令：\nbash\ngitremoteaddorigin远程仓库地址\n其中，origin是远程仓库的别名，可以根据实际情况自行命名。\n推送到远程仓库\n将本地提交推送到远程仓库可以使用以下命令：\nbash\ngitpushorigin分支名\n例如，将本地的master分支推送到远程仓库可以使用：\nbash\ngitpushorig

In [21]:
# 最后选择最相似的文档, 交给LLM作为可参考上下文
best_content = rerank_content[0]
print(chat.chat(question, [], best_content))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


好的，根据上下文，你可以使用以下步骤在Git中新建分支：
1. 使用`git branch`命令创建新分支：
```bash
git branch <分支名>
```
2. 使用`git checkout`命令切换到新分支：
```bash
git checkout <分支名>
```
3. 创建新分支并立即切换到该分支：
```bash
git checkout -b <分支名>
```
关于分支策略，GitFlow和GitHubFlow都是常见的策略。GitFlow定义了一个围绕项目发布的分支模型，包括功能分支、发布分支、维护分支等。GitHubFlow则更加简单灵活，适合持续交付的项目。在GitHubFlow中，master分支通常是稳定的，Lj随时可以部署。所有新的开发都在基于master的特性分支上进行，一旦完成就可以合并回master。

解决冲突方面，你可以使用`git merge`命令将两个分支合并，但可能会遇到冲突。
